In [151]:
import pandas as pd
df = pd.read_csv('../csv_data/IFM/IFM - invoice_29050823.csv')

In [152]:
from pdf2df import convert_to_df
df = convert_to_df('../multipdf/IFM - 29371311.pdf')
df

{'format': 'PDF 1.4', 'title': 'Untitled', 'author': 'Unknown', 'subject': '', 'keywords': '', 'creator': 'Unknown', 'producer': 'XEP 4.11 build 20071214', 'creationDate': 'D:20210226012301Z', 'modDate': 'D:20210308072617Z', 'trapped': '', 'encryption': None}


Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.


,TEXT,X1,Y1,X2,Y2,block_no,line_no,page_no
0,ifm,59,140,68,148,0,0,1
1,electronic,70,140,100,148,0,0,1
2,(Ireland),102,140,128,148,0,0,1
3,Ltd,130,140,139,148,0,0,1
4,•,141,140,144,148,0,0,1
...,...,...,...,...,...,...,...,...
456,D01370_IE,470,134,510,143,11,1,2
457,Customer,405,125,440,134,11,2,2
458,no.,442,125,453,134,11,2,2
459,25.02.2021,470,144,510,153,12,0,2


In [153]:
def find_text(temp_df, text,case=False):
    location = temp_df[temp_df['TEXT'].str.contains(text,case=case, na=False)].index.tolist()
    return location

def find_next_word(temp_df, text, nearest_text, case=False):
    pos = find_text(temp_df,text,case=case)
    pos = [item +1 for item in pos]
#     print(pos)
    new_pos = list()
    for item in pos:
#         temp = df.loc[]
        temp_value= temp_df.loc[item]['TEXT']
#         print(temp_value.lower())
       
        if temp_value.lower().__contains__(nearest_text.lower()):
            new_pos.append(item-1)
        
    return new_pos

def rearrange(df):
    for i in range(1, len(df)):
        temp=df.loc[i,'Y1'] - df.loc[i-1, 'Y1']
        if -5 < temp < 10:
            df.loc[i,'Y1'] = df.loc[i-1,'Y1']
    
    return df.sort_values(by=['page_no','Y1','X1']).reset_index(drop=True)

In [154]:
df_sort= df.sort_values(by=['page_no','Y1','X1']).reset_index(drop=True)
df = rearrange(df_sort)
df

,TEXT,X1,Y1,X2,Y2,block_no,line_no,page_no
0,Invoice,54,56,105,72,14,0,1
1,ifm,59,140,68,148,0,0,1
2,Clondalkin,59,140,91,156,0,1,1
3,electronic,70,140,100,148,0,0,1
4,•,93,140,96,156,0,1,1
...,...,...,...,...,...,...,...,...
456,have,282,813,296,826,7,4,2
457,been,298,813,312,826,7,4,2
458,fully,313,813,326,826,7,4,2
459,paid,327,813,340,826,7,4,2


In [155]:
page_no = df['page_no'].unique()
page_no

array([1, 2], dtype=int64)

In [156]:
pos = find_next_word(df,'Invoice', 'no.')
invoice_no = df[pos[0]+2:pos[0]+3]['TEXT'].values[0]
invoice_no

'29371311'

In [157]:
pos_date =find_text(df,'date')[0]
date = df[pos_date+1:pos_date+2]['TEXT'].values[0]
date

'25.02.2021'

In [158]:
result = []
for page in page_no:
    mydf = df[df['page_no']==page].reset_index(drop=True)
    get_result(mydf,result)
len(result)

{'desc': ' ADOGH050MSS0015H05 Country of origin Customs number tariff',
 'item': '0030',
 'part': 'EVC197',
 'qty': '5',
 'total': '96.50',
 'unit': '19.30'}
{'desc': ' ADOGH050MSS0005H05',
 'item': '0040',
 'part': 'EVC071',
 'qty': '1',
 'total': '12.67',
 'unit': '12.67'}
{'desc': ' ADOGH050MSS0015H05 Country of origin Customs number tariff',
 'item': '0030',
 'part': 'EVC197',
 'qty': '5',
 'total': '96.50',
 'unit': '19.30'}
{'desc': ' ADOGH050MSS0005H05',
 'item': '0040',
 'part': 'EVC071',
 'qty': '1',
 'total': '12.67',
 'unit': '12.67'}


4

In [136]:
original_df = df.copy()
df = original_df[original_df.page_no==2]
df

,TEXT,X1,Y1,X2,Y2,block_no,line_no,page_no


In [138]:
table_start = find_text(df, '00')
print(table_start)
table_end = find_text(df, 'Country')
print(table_end)
table_data = df[table_start[0]: table_end[-1]]
table_data

[]
[]


IndexError: list index out of range

In [132]:
from pprint import pprint
msg = dict()
for index, row in table_data.iterrows():

    if (50 <row['X1'] < 76):
        if 'part' in msg :
            result.append(msg)
            pprint(msg)
        msg =dict()
        msg['item'] = row['TEXT']
        msg['part'] = ''
        msg['qty'] = ''
        msg['desc'] = ''

    if ('part' in msg) and (204 < row['X1'] < 300) and (msg['part']==''):
            msg['part'] = row['TEXT']

    elif ('desc' in msg) and (204 < row['X1'] < 270):
        msg['desc'] = msg['desc'] +" "+row['TEXT']


    elif ('qty' in msg) and (380 < row['X1'] < 397):
        msg['qty'] = row['TEXT']
        msg['unit'] = 0

    elif ('unit' in msg) and (420 < row['X1'] < 482):
        msg['unit'] = row['TEXT']
        msg['total'] = 0

    elif ('total' in msg) and (500 < row['X1'] < 530):
        msg['total'] = row['TEXT']


result.append(msg)
pprint(msg)


{'desc': ' ADOGH050MSS0015H05 Country of origin Customs number tariff',
 'item': '0030',
 'part': 'EVC197',
 'qty': '5',
 'total': '96.50',
 'unit': '19.30'}
{'desc': ' ADOGH050MSS0005H05',
 'item': '0040',
 'part': 'EVC071',
 'qty': '1',
 'total': '12.67',
 'unit': '12.67'}


In [ ]:
from pprint import pprint
msg = dict()
for index, row in table_data.iterrows():

    if (50 <row['X1'] < 76):
        if 'part' in msg :
            result.append(msg)
            pprint(msg)
        msg =dict()
        msg['item'] = row['TEXT']
        msg['part'] = ''
        msg['qty'] = ''
        msg['desc'] = ''

    if ('part' in msg) and (204 < row['X1'] < 300):
        if msg['part']=='':
            msg['part'] = row['TEXT']

    if ('desc' in msg) and (133 < row['X1'] < 250):
        msg['desc'] = msg['desc'] +" "+row['TEXT']


    if ('qty' in msg) and (380 < row['X1'] < 397):
        msg['qty'] = row['TEXT']
        msg['unit'] = 0

    if ('unit' in msg) and (420 < row['X1'] < 482):
        msg['unit'] = row['TEXT']
        msg['total'] = 0

    if ('total' in msg) and (500 < row['X1'] < 530):
        msg['total'] = row['TEXT']


result.append(msg)
pprint(msg)


In [148]:
def get_result(df,result):
    
    from pprint import pprint
    msg = dict()
    for index, row in table_data.iterrows():

        if (50 <row['X1'] < 76):
            if 'part' in msg :
                result.append(msg)
                pprint(msg)
            msg =dict()
            msg['item'] = row['TEXT']
            msg['part'] = ''
            msg['qty'] = ''
            msg['desc'] = ''

        if ('part' in msg) and (204 < row['X1'] < 300) and (msg['part']==''):
                msg['part'] = row['TEXT']

        elif ('desc' in msg) and (204 < row['X1'] < 270):
            msg['desc'] = msg['desc'] +" "+row['TEXT']


        elif ('qty' in msg) and (380 < row['X1'] < 397):
            msg['qty'] = row['TEXT']
            msg['unit'] = 0

        elif ('unit' in msg) and (420 < row['X1'] < 482):
            msg['unit'] = row['TEXT']
            msg['total'] = 0

        elif ('total' in msg) and (500 < row['X1'] < 530):
            msg['total'] = row['TEXT']


    result.append(msg)
    pprint(msg)
        

In [149]:
result

[]